In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE150734"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE150734"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE150734.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE150734.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE150734.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prognostic Liver Signature profiles in biopsy tissues from non-alcoholic fatty liver disease (NAFLD) patients in the U.S."
!Series_summary	"Background/Aims: There is a major unmet need to assess prognostic impact of anti-fibrotics in clinical trials due to the slow rate of liver fibrosis progression. We aimed to develop a surrogate biomarker to predict future fibrosis progression.  Methods: A Fibrosis Progression Signature (FPS) was defined to predict fibrosis progression within 5 years in HCV and NAFLD patients with no to minimal fibrosis at baseline (n=421), and validated in an independent NAFLD cohort (n=78). The FPS was used to assess response to 13 candidate anti-fibrotics in organotypic ex vivo cultures of clinical fibrotic liver tissues (n=78), and cenicriviroc in NASH patients enrolled in a clinical trial (n=19, NCT02217475). A serum-protein-based surrogate FPS (FPSec) was developed and technically evaluated in a liver disease patient coho

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background info, this appears to be gene expression data from liver biopsies
# Series title mentions "Liver Signature profiles" and overall design mentions "Gene expression profiling"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Liver_cirrhosis), we can infer from "fibrosis stage" in the sample characteristics
trait_row = 0  # fibrosis stage is in row 0

# Age is not available in the sample characteristics dictionary
age_row = None

# Gender is not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert liver fibrosis stage to binary indication of cirrhosis.
    In fibrosis staging, stages 0-3 are non-cirrhotic, while stage 4 is cirrhotic.
    However, this dataset only shows stages 0 and 1, indicating no cirrhosis.
    """
    if value is None:
        return None
    
    # Extract the value after ":"
    if ":" in value:
        stage_str = value.split(":")[1].strip()
    else:
        stage_str = value.strip()
    
    try:
        # Convert to integer
        stage = int(stage_str)
        # In liver fibrosis staging, stage 4 is cirrhosis
        # Stages 0-3 are non-cirrhotic
        # This dataset only shows stages 0 and 1
        return 1 if stage == 4 else 0  # 1 for cirrhosis, 0 for non-cirrhosis
    except ValueError:
        return None

# No conversion functions needed for age and gender as they are not available
convert_age = None
convert_gender = None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Define clinical_data - use the actual dictionary from the output
    sample_characteristics_dict = {0: ['fibrosis stage: 0', 'fibrosis stage: 1'], 
                                  1: ['pls risk prediction: High', 'pls risk prediction: Intermediate', 'pls risk prediction: Low']}
    
    # Create clinical_data DataFrame with the right structure
    clinical_data = pd.DataFrame()
    for idx, values in sample_characteristics_dict.items():
        clinical_data[idx] = pd.Series(values)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{0: [0.0], 1: [nan]}
Clinical data saved to ../../output/preprocess/Liver_cirrhosis/clinical_data/GSE150734.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE150734/GSE150734_series_matrix.txt.gz
Gene data shape: (192, 309)
First 20 gene/probe identifiers:
Index(['AARS', 'ABLIM1', 'ACOT2', 'ACSM3', 'ACTR2', 'ADD3', 'ADH5', 'ADH6',
       'ADRA2B', 'AEBP1', 'AKAP13', 'AKR1A1', 'AKR1D1', 'ALAS1', 'ALDH9A1',
       'ANKRD46', 'ANXA1', 'ANXA3', 'AOX1', 'AP1B1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers in the gene expression data, these appear to be standard human gene symbols
# (like AARS, ABLIM1, ACOT2, etc.) which are official gene symbols.
# They don't require mapping to different identifiers.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (192, 309)
Gene data shape after normalization: (191, 309)
Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE150734.csv
Original clinical data preview:
         !Sample_geo_accession                 GSM4557370  \
0  !Sample_characteristics_ch1          fibrosis stage: 0   
1  !Sample_characteristics_ch1  pls risk prediction: High   

                          GSM4557371                         GSM4557372  \
0                  fibrosis stage: 0                  fibrosis stage: 0   
1  pls risk prediction: Intermediate  pls risk prediction: Intermediate   

                  GSM4557373                         GSM4557374  \
0          fibrosis stage: 0                  fibrosis stage: 0   
1  pls risk prediction: High  pls risk prediction: Intermediate   

                  GSM4557375                 GSM4557376  \
0          fibrosis stage: 0          fibrosis stage: 0   
1  pls risk prediction: High  pls ri

Quartiles for 'Liver_cirrhosis':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Liver_cirrhosis' in this dataset is severely biased.

Data shape after removing biased features: (309, 192)


Dataset is not usable for analysis. No linked data file saved.
